<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/QMC_Multimodal_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

In [2]:
import tensorflow as tf
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [3]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape((60000,784))
X_test = X_test.reshape((10000,784))

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

transformer = OneHotEncoder(sparse=False)
y_train_bin = transformer.fit_transform(y_train[:, np.newaxis])
y_test_bin = transformer.fit_transform(y_test[:, np.newaxis])

print("shape X_train : ", X_train.shape)
print("shape y_train : ", y_train.shape)
print("shape X_test : ", X_test.shape)
print("shape y_test : ", y_test.shape)

shape X_train :  (60000, 784)
shape y_train :  (60000,)
shape X_test :  (10000, 784)
shape y_test :  (10000,)


## Baseline

In [4]:
BATCH_SIZE = 256

model = tf.keras.models.Sequential([
tf.keras.layers.Dense(128,activation='relu'),
tf.keras.layers.Dense(10),
tf.keras.layers.Softmax()
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)
EPOCHS = 10
  
history = model.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 1s 3ms/step - loss: 0.8690 - categorical_accuracy: 0.7577 - val_loss: 0.2460 - val_categorical_accuracy: 0.9333
Epoch 2/10
188/188 [==============================] - 1s 3ms/step - loss: 0.2367 - categorical_accuracy: 0.9331 - val_loss: 0.1793 - val_categorical_accuracy: 0.9519
Epoch 3/10
188/188 [==============================] - 0s 3ms/step - loss: 0.1683 - categorical_accuracy: 0.9523 - val_loss: 0.1618 - val_categorical_accuracy: 0.9547
Epoch 4/10
188/188 [==============================] - 1s 3ms/step - loss: 0.1322 - categorical_accuracy: 0.9621 - val_loss: 0.1334 - val_categorical_accuracy: 0.9614
Epoch 5/10
188/188 [==============================] - 0s 3ms/step - loss: 0.1078 - categorical_accuracy: 0.9706 - val_loss: 0.1207 - val_categorical_accuracy: 0.9651
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.0927 - categorical_accuracy: 0.9739 - val_loss: 0.1116 - val_categorical_accuracy: 0.9665
Epoc

## Half model

In [32]:
BATCH_SIZE = 256
def create_model(input_dim,
               output_dim,
               num_rff = 512,
               gamma = 2**-5,
               n_comp = 80,
               random_state = 0):
    inputs = tf.keras.Input(shape=(input_dim,))
    fm_x1 = layers.QFeatureMapRFF(input_dim, dim=num_rff , gamma=gamma, random_state=random_state)
    psi_x = fm_x1(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=num_rff, dim_y=output_dim, n_comp=n_comp)
    rho_y = qmdmc(rho_x)
    y_w = rho_y[:, 0, :] # shape (b, d_in)
    y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
    probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
    qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
    return qmdmclf

def create_model_rho(input_dim,
               output_dim,
               num_rff = 512,
               gamma = 2**-5,
               n_comp = 80,
               random_state = 0):
    inputs = tf.keras.Input(shape=(input_dim,))
    fm_x1 = layers.QFeatureMapRFF(input_dim, dim=num_rff , gamma=gamma, random_state=random_state)
    psi_x = fm_x1(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=num_rff, dim_y=output_dim, n_comp=n_comp)
    rho_y = qmdmc(rho_x)
    qmdmclf = tf.keras.Model(inputs=inputs, outputs=rho_y)
    return qmdmclf


In [36]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
qmdmclf_1 = create_model(784 // 2, 10, num_rff=256, n_comp=20)
qmdmclf_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
BATCH_SIZE = 256  
history = qmdmclf_1.fit(X_train[:, 0:784 // 2], y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 2s 8ms/step - loss: 1.5393 - accuracy: 0.5709 - val_loss: 0.6130 - val_accuracy: 0.8671
Epoch 2/10
188/188 [==============================] - 1s 7ms/step - loss: 0.5562 - accuracy: 0.8702 - val_loss: 0.4531 - val_accuracy: 0.8948
Epoch 3/10
188/188 [==============================] - 1s 8ms/step - loss: 0.4273 - accuracy: 0.8983 - val_loss: 0.4118 - val_accuracy: 0.9048
Epoch 4/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3795 - accuracy: 0.9054 - val_loss: 0.3874 - val_accuracy: 0.9099
Epoch 5/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3480 - accuracy: 0.9147 - val_loss: 0.3701 - val_accuracy: 0.9134
Epoch 6/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3251 - accuracy: 0.9194 - val_loss: 0.3535 - val_accuracy: 0.9161
Epoch 7/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3091 - accuracy: 0.9239 - val_loss: 0.3455 - val_accuracy: 0.9196
Epoch 

In [37]:
qmdmclf_2 = create_model(784 // 2, 10, num_rff=256, n_comp=20)
qmdmclf_2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

history = qmdmclf_2.fit(X_train[:, 784 // 2:], y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 2s 8ms/step - loss: 1.0416 - accuracy: 0.6906 - val_loss: 0.4646 - val_accuracy: 0.8661
Epoch 2/10
188/188 [==============================] - 1s 7ms/step - loss: 0.4343 - accuracy: 0.8704 - val_loss: 0.4199 - val_accuracy: 0.8758
Epoch 3/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3798 - accuracy: 0.8849 - val_loss: 0.3971 - val_accuracy: 0.8826
Epoch 4/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3535 - accuracy: 0.8903 - val_loss: 0.3752 - val_accuracy: 0.8857
Epoch 5/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3334 - accuracy: 0.8950 - val_loss: 0.3667 - val_accuracy: 0.8878
Epoch 6/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3197 - accuracy: 0.9021 - val_loss: 0.3625 - val_accuracy: 0.8897
Epoch 7/10
188/188 [==============================] - 1s 7ms/step - loss: 0.3104 - accuracy: 0.9062 - val_loss: 0.3570 - val_accuracy: 0.8903
Epoch 

## Multimodal model

In [55]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
clf_2 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
rho_1 = clf_1(inputs[:, :784 // 2])
rho_2 = clf_2(inputs[:, 784 // 2:])
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_multim = tf.keras.Model(inputs=inputs, outputs=probs)



In [56]:
qm_multim.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

EPOCHS = 10
history = qm_multim.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/10
188/188 [==============================] - 25s 125ms/step - loss: 0.8980 - accuracy: 0.7620 - val_loss: 0.2550 - val_accuracy: 0.9473
Epoch 2/10
188/188 [==============================] - 23s 123ms/step - loss: 0.2256 - accuracy: 0.9552 - val_loss: 0.2184 - val_accuracy: 0.9541
Epoch 3/10
188/188 [==============================] - 23s 122ms/step - loss: 0.1875 - accuracy: 0.9627 - val_loss: 0.1994 - val_accuracy: 0.9578
Epoch 4/10
188/188 [==============================] - 23s 124ms/step - loss: 0.1632 - accuracy: 0.9695 - val_loss: 0.1887 - val_accuracy: 0.9601
Epoch 5/10
188/188 [==============================] - 23s 123ms/step - loss: 0.1459 - accuracy: 0.9740 - val_loss: 0.1811 - val_accuracy: 0.9610
Epoch 6/10
188/188 [==============================] - 23s 123ms/step - loss: 0.1322 - accuracy: 0.9776 - val_loss: 0.1777 - val_accuracy: 0.9631
Epoch 7/10
188/188 [==============================] - 23s 123ms/step - loss: 0.1234 - accuracy: 0.9802 - val_loss: 0.1740 - val_ac

In [10]:
qm_multim.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem_8 (Sli (None, 392)          0           input_3[0][0]                    
__________________________________________________________________________________________________
tf.__operators__.getitem_9 (Sli (None, 392)          0           input_3[0][0]                    
__________________________________________________________________________________________________
model_2 (Functional)            (None, 11, 10)       103278      tf.__operators__.getitem_8[0][0] 
____________________________________________________________________________________________

## Noisy dataset

In [57]:
X_test_noise = np.array(X_test)
X_test_noise[:, :784 // 2] =  np.random.uniform(size=X_test.shape)[:, :784 // 2]
model.evaluate(X_test_noise, y_test_bin)

313/313 [==============================] - 0s 482us/step - loss: 9.7003 - categorical_accuracy: 0.2244


[9.70030689239502, 0.22439999878406525]

In [58]:
qm_multim.evaluate(X_test_noise, y_test_bin)

313/313 [==============================] - 2s 6ms/step - loss: 1.6451 - accuracy: 0.3966


[1.6450669765472412, 0.39660000801086426]

## Partial model

In [60]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
rho_1 = clf_1(inputs[:, :784 // 2])
rho_in = tf.keras.layers.concatenate((tf.ones((1, 256)) / 256., tf.eye(256)), axis = 0)
rho_in = tf.expand_dims(rho_in, axis=0)
clf_2 = layers.QMClassifSDecompFDMatrix(dim_x=256, dim_y=10, n_comp=20, name="clf_2")
clf_2(rho_in)
clf_2.set_weights(qm_multim.layers[4].layers[-1].get_weights())
rho_2 = clf_2(rho_in)
#rho_2 = tf.keras.layers.concatenate((rho_2, tf.zeros((11, 10))), axis=1)
rho_2 = tf.broadcast_to(rho_2, shape=tf.shape(rho_1))
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_partial = tf.keras.Model(inputs=inputs, outputs=probs)
qm_partial.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

In [61]:
qm_partial.layers

In [62]:
qm_partial.layers[2].set_weights(qm_multim.layers[3].get_weights())
qm_partial.layers[6].set_weights(qm_multim.layers[6].get_weights())

In [63]:
qm_partial.evaluate(X_test, y_test_bin)

313/313 [==============================] - 2s 6ms/step - loss: 1.1990 - accuracy: 0.4815


[1.2016475200653076, 0.4797999858856201]

## Multimodal with fixed layers

In [64]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
clf_2 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
rho_1 = clf_1(inputs[:, :784 // 2])
rho_2 = clf_2(inputs[:, 784 // 2:])
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_multim = tf.keras.Model(inputs=inputs, outputs=probs)

qm_multim.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

qm_multim.layers[3].set_weights(qmdmclf_1.get_weights())
qm_multim.layers[3].trainable = False
qm_multim.layers[4].set_weights(qmdmclf_2.get_weights())
qm_multim.layers[4].trainable = False


In [65]:

EPOCHS = 3
history = qm_multim.fit(X_train, y_train_bin, batch_size=BATCH_SIZE,
                    epochs=EPOCHS, validation_split=0.2)

Epoch 1/3
188/188 [==============================] - 25s 125ms/step - loss: 0.6610 - accuracy: 0.8872 - val_loss: 0.1700 - val_accuracy: 0.9712
Epoch 2/3
188/188 [==============================] - 23s 123ms/step - loss: 0.1328 - accuracy: 0.9821 - val_loss: 0.1558 - val_accuracy: 0.9718
Epoch 3/3
188/188 [==============================] - 23s 124ms/step - loss: 0.1120 - accuracy: 0.9859 - val_loss: 0.1513 - val_accuracy: 0.9715


In [66]:
inputs = tf.keras.Input(shape=(784,))
clf_1 = create_model_rho(784 // 2, 10, num_rff=256, n_comp=20)
rho_1 = clf_1(inputs[:, :784 // 2])
rho_in = tf.keras.layers.concatenate((tf.ones((1, 256)) / 256., tf.eye(256)), axis = 0)
rho_in = tf.expand_dims(rho_in, axis=0)
clf_2 = layers.QMClassifSDecompFDMatrix(dim_x=256, dim_y=10, n_comp=20, name="clf_2")
clf_2(rho_in)
clf_2.set_weights(qm_multim.layers[4].layers[-1].get_weights())
rho_2 = clf_2(rho_in)
#rho_2 = tf.keras.layers.concatenate((rho_2, tf.zeros((11, 10))), axis=1)
rho_2 = tf.broadcast_to(rho_2, shape=tf.shape(rho_1))
prod = layers.DMCrossProduct()([rho_1, rho_2])
qmdmc = layers.QMClassifSDecompFDMatrix(dim_x=100, dim_y=10, n_comp=40)
rho_y = qmdmc(prod)
y_w = rho_y[:, 0, :] # shape (b, d_in)
y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v), optimize='optimal')
qm_partial = tf.keras.Model(inputs=inputs, outputs=probs)
qm_partial.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

In [67]:
qm_partial.layers[2].set_weights(qm_multim.layers[3].get_weights())
qm_partial.layers[6].set_weights(qm_multim.layers[6].get_weights())

In [71]:
qm_partial.evaluate(X_test, y_test_bin)

313/313 [==============================] - 2s 7ms/step - loss: 1.0419 - accuracy: 0.6605


[1.0419251918792725, 0.6604999899864197]

In [69]:
qm_multim.evaluate(X_test_noise, y_test_bin)

313/313 [==============================] - 2s 6ms/step - loss: 0.8688 - accuracy: 0.7153


[0.868797779083252, 0.7153000235557556]